In [50]:
import requests
import pandas
import numpy
from arcgis.features import GeoAccessor
from arcpy.sa import *
import arcpy

In [51]:
hour = 12 #Hour to retrieve weather data for, in 24 hour time. Midnight uses a value of 24 for the previous day.

#Date
year = 2024
month = 3
day = 25

In [52]:
#File paths
stem_path = r'C:/Users/tjjoh/Documents/GIS5571'

gdb_path = stem_path + r'/FinalProject_GDB.gdb'
temp_fc_path = gdb_path + r'/temp'
humid_fc_path = gdb_path + r'/humid'
pres_fc_path = gdb_path + r'/pres'
wind_fc_path = gdb_path + r'/wind'

idw1_temp_path = gdb_path + r'/temp_IDW1'
idw2_temp_path = gdb_path + r'/temp_IDW2'
nn_temp_path = gdb_path + r'/temp_nn'
spline_temp_path = gdb_path + r'/temp_spline'
kriging_temp_path = gdb_path + r'/temp_kriging'

idw1_humid_path = gdb_path + r'/humid_IDW1'
idw2_humid_path = gdb_path + r'/humid_IDW2'
nn_humid_path = gdb_path + r'/humid_nn'
spline_humid_path = gdb_path + r'/humid_spline'
kriging_humid_path = gdb_path + r'/humid_kriging'

idw1_pres_path = gdb_path + r'/pres_IDW1'
idw2_pres_path = gdb_path + r'/pres_IDW2'
nn_pres_path = gdb_path + r'/pres_nn'
spline_pres_path = gdb_path + r'/pres_spline'
kriging_pres_path = gdb_path + r'/pres_kriging'

idw1_windN_path = gdb_path + r'/windN_IDW1'
idw2_windN_path = gdb_path + r'/windN_IDW2'
nn_windN_path = gdb_path + r'/windN_nn'
spline_windN_path = gdb_path + r'/windN_spline'
kriging_windN_path = gdb_path + r'/windN_kriging'

idw1_windE_path = gdb_path + r'/windE_IDW1'
idw2_windE_path = gdb_path + r'/windE_IDW2'
nn_windE_path = gdb_path + r'/windE_nn'
spline_windE_path = gdb_path + r'/windE_spline'
kriging_windE_path = gdb_path + r'/windE_kriging'

idw1_wind_path = gdb_path + r'/wind_IDW1'
idw2_wind_path = gdb_path + r'/wind_IDW2'
nn_wind_path = gdb_path + r'/wind_nn'
spline_wind_path = gdb_path + r'/wind_spline'
kriging_wind_path = gdb_path + r'/wind_kriging'

idw1_winddir_path = gdb_path + r'/winddir_IDW1'
idw2_winddir_path = gdb_path + r'/winddir_IDW2'
nn_winddir_path = gdb_path + r'/winddir_nn'
spline_winddir_path = gdb_path + r'/winddir_spline'
kriging_winddir_path = gdb_path + r'/winddir_kriging'

In [53]:
utc = str((hour + 5) % 24).zfill(2) #Convert time to UTC. This is valid only during daylight savings time

#Convert date and time to strings
yr = str(year)
mn = str(month).zfill(2)
dy_ndawn = str(day).zfill(2)
if hour < 19:
    dy_noaa = str(day).zfill(2)
else:
    dy_noaa = str(day + 1).zfill(2)
hr = str(hour).zfill(2)

#URLs for API calls
noaa_url = r'https://api.synopticlabs.org/v2/stations/nearesttime?token=d8c6aee36a994f90857925cea26934be&bbox=-105%2C42%2C-86.5%2C49.5&minmax=1&minmaxtype=local&units=temp%7Cf%2Cspeed%7Cmph&attime=' + yr + mn + dy_noaa + utc + '00&within=90&status=active&qc_checks=all'
ndawn_url = 'https://ndawn.ndsu.nodak.edu/table.csv?station>0&variable=hdt&variable=hdbp&variable=hdrh&variable=hdws&variable=hdwd&ttype=hourly&quick_pick=&begin_date=' + yr + '-' + mn + '-' + dy_ndawn + '&end_date=' + yr + '-' + mn + '-' + dy_ndawn

**Prepare NOAA Data**

In [54]:
#Make API request
noaa_response = requests.get(noaa_url)

In [55]:
#Convert NOAA data to dataframe
noaa_df = pandas.json_normalize(noaa_response.json()['STATION'])

In [56]:
#Rename columns
noaa_df = noaa_df.rename(columns = {'OBSERVATIONS.air_temp_value_1.value' : 'Temperature',
                                    'OBSERVATIONS.relative_humidity_value_1.value' : 'Humidity',
                                    'OBSERVATIONS.wind_speed_value_1.value' : 'WindSpeed',
                                    'OBSERVATIONS.wind_direction_value_1.value' : 'WindDirection',
                                    'OBSERVATIONS.altimeter_value_1.value' : 'Pressure',
                                    'NAME' : 'Name',
                                    'ELEVATION' : 'Elevation',
                                    'LATITUDE' : 'Latitude',
                                    'LONGITUDE' : 'Longitude',
                                    'STATE' : 'State',
                                    'TIMEZONE' : 'Timezone'
                                   })

In [57]:
#Keep only desired columns
noaa_df = noaa_df[['ID','Name','Elevation','Latitude','Longitude','State','Timezone',
                   'Temperature','Humidity','WindSpeed','WindDirection','Pressure'
                  ]]

In [58]:
#Convert pressure units to mbar, the same units that NDAWN uses
noaa_df['Pressure'] = noaa_df['Pressure'] / 100

**Prepare NDAWN Data**

In [59]:
#Read data into a pandas dataframe
ndawn_df = pandas.read_csv(ndawn_url, skiprows = [0,1,2,4])

In [60]:
#Choose only the specified hour
ndawn_df = ndawn_df[ndawn_df['Hour'] == hour * 100]

In [61]:
#Rename columns
ndawn_df = ndawn_df.rename(columns = {'Station Name' : 'Name',
                                      'Avg Air Temp' : 'Temperature',
                                      'Avg Baro Press' : 'Pressure',
                                      'Avg Rel Hum' : 'Humidity',
                                      'Avg Wind Speed' : 'WindSpeed',
                                      'Avg Wind Dir' : 'WindDirection'
                                     })

In [62]:
#Keep only desired columns
ndawn_df = ndawn_df[['Name','Elevation','Latitude','Longitude',
                     'Temperature','Humidity','WindSpeed','WindDirection','Pressure'
                    ]]

**Combine Data Frames**

In [63]:
#Concatenate data frames
weather_df = pandas.concat([noaa_df, ndawn_df], ignore_index = True)

In [64]:
#Convert to a spatially-enabled dataframe
weather_sedf = GeoAccessor.from_xy(weather_df, x_column = 'Longitude', y_column = 'Latitude', sr = 4326)

In [65]:
#Calculate z-scores for weather quantities
weather_sedf['Temp_z'] = abs((weather_sedf['Temperature'] - weather_sedf['Temperature'].mean()) / weather_sedf['Temperature'].std())
weather_sedf['Humid_z'] = abs((weather_sedf['Humidity'] - weather_sedf['Humidity'].mean()) / weather_sedf['Humidity'].std())
weather_sedf['Pres_z'] = abs((weather_sedf['Pressure'] - weather_sedf['Pressure'].mean()) / weather_sedf['Pressure'].std())
weather_sedf['Wind_z'] = abs((weather_sedf['WindSpeed'] - weather_sedf['WindSpeed'].mean()) / weather_sedf['WindSpeed'].std())

In [66]:
#Remove instances with high z-score because they are likely inaccurate
temp_sedf = weather_sedf[weather_sedf['Temp_z'] < 3]
humid_sedf = weather_sedf[weather_sedf['Humid_z'] < 3]
pres_sedf = weather_sedf[weather_sedf['Pres_z'] < 3]
wind_sedf = weather_sedf[weather_sedf['Wind_z'] < 3]

In [67]:
#Deconstruct wind into North and East components
wind_sedf = wind_sedf[wind_sedf['WindDirection'].notna()]

wind_sedf['Wind_N'] = wind_sedf['WindSpeed'] * numpy.cos(numpy.radians(wind_sedf['WindDirection']))
wind_sedf['Wind_E'] = wind_sedf['WindSpeed'] * numpy.sin(numpy.radians(wind_sedf['WindDirection']))

In [68]:
#Convert spatially enabled dataframes to feature classes
temp_fc = temp_sedf.spatial.to_featureclass(location = temp_fc_path)
humid_fc = humid_sedf.spatial.to_featureclass(location = humid_fc_path)
pres_fc = pres_sedf.spatial.to_featureclass(location = pres_fc_path)
wind_fc = wind_sedf.spatial.to_featureclass(location = wind_fc_path)

**Interpolation**

In [70]:
#create lists of variables we want to interpolate, interpolation methods, and file paths for easier indexing
quantities = ['Temperature','Humidity','Pressure','Wind_N','Wind_E']
methods = ['IDW1', 'IDW2', 'NN', 'Spline', 'Kriging']
input_paths = [temp_fc_path, humid_fc_path, pres_fc_path, wind_fc_path, wind_fc_path]

output_paths_idw1 = [idw1_temp_path, idw1_humid_path, idw1_pres_path, idw1_windN_path, idw1_windE_path, idw1_wind_path, idw1_winddir_path]
output_paths_idw2 = [idw2_temp_path, idw2_humid_path, idw2_pres_path, idw2_windN_path, idw2_windE_path, idw2_wind_path, idw2_winddir_path]
output_paths_nn = [nn_temp_path, nn_humid_path, nn_pres_path, nn_windN_path, nn_windE_path, nn_wind_path, nn_winddir_path]
output_paths_spline = [spline_temp_path, spline_humid_path, spline_pres_path, spline_windN_path, spline_windE_path, spline_wind_path, spline_winddir_path]
output_paths_kriging = [kriging_temp_path, kriging_humid_path, kriging_pres_path, kriging_windN_path, kriging_windE_path, kriging_wind_path, kriging_winddir_path]

output_paths = [output_paths_idw1, output_paths_idw2, output_paths_nn, output_paths_spline, output_paths_kriging]

In [71]:
#Iterate over weather variables that we want to interpolate
for i in range(len(quantities)):
    #First Order IDW
    with arcpy.EnvManager(scratchWorkspace = gdb_path):
        IDW1 = arcpy.sa.Idw(
            in_point_features = input_paths[i],
            z_field = quantities[i],
            cell_size = 0.021010908,
            power = 1,
            search_radius = 'VARIABLE 12',
            in_barrier_polyline_features = None
        )
        IDW1.save(output_paths[0][i] + '_' + hr)
    print(f'{quantities[i]} {methods[0]} complete')
    
    #Second Order IDW
    with arcpy.EnvManager(scratchWorkspace = gdb_path):
        IDW2 = arcpy.sa.Idw(
            in_point_features = input_paths[i],
            z_field = quantities[i],
            cell_size = 0.021010908,
            power = 2,
            search_radius = 'VARIABLE 12',
            in_barrier_polyline_features = None
        )
        IDW2.save(output_paths[1][i] + '_' + hr)
    print(f'{quantities[i]} {methods[1]} complete')
    
    #Natural neighbor
    with arcpy.EnvManager(scratchWorkspace = gdb_path):
        NN = arcpy.sa.NaturalNeighbor(
            in_point_features = input_paths[i],
            z_field = quantities[i],
            cell_size = 0.021010908
        )
        NN.save(output_paths[2][i] + '_' + hr)
    print(f'{quantities[i]} {methods[2]} complete')
    
    #Spline interpolation
    with arcpy.EnvManager(scratchWorkspace = gdb_path):
        Spline = arcpy.sa.Spline(
            in_point_features = input_paths[i],
            z_field = quantities[i],
            cell_size = 0.021010908,
            spline_type = 'REGULARIZED',
            weight = 0.1,
            number_points = 12
        )
        Spline.save(output_paths[3][i] + '_' + hr)
    print(f'{quantities[i]} {methods[3]} complete')
    
    #Kriging
    with arcpy.EnvManager(scratchWorkspace = gdb_path):
        Krig = arcpy.sa.Kriging(
            in_point_features = input_paths[i],
            z_field = quantities[i],
            kriging_model = 'Spherical # # # #',
            cell_size = 0.021010908,
            search_radius = 'VARIABLE 12',
            out_variance_prediction_raster = None
        )
        Krig.save(output_paths[4][i] + '_' + hr)
    print(f'{quantities[i]} {methods[4]} complete')

Temperature IDW1 complete
Temperature IDW2 complete
Temperature NN complete
Temperature Spline complete
Temperature Kriging complete
Humidity IDW1 complete
Humidity IDW2 complete
Humidity NN complete
Humidity Spline complete
Humidity Kriging complete
Pressure IDW1 complete
Pressure IDW2 complete
Pressure NN complete
Pressure Spline complete
Pressure Kriging complete
Wind_N IDW1 complete
Wind_N IDW2 complete
Wind_N NN complete
Wind_N Spline complete
Wind_N Kriging complete
Wind_E IDW1 complete
Wind_E IDW2 complete
Wind_E NN complete
Wind_E Spline complete
Wind_E Kriging complete


In [72]:
for i in range(len(methods)):
    #Assign north-south and east-west components of wind to a variable
    NORTH = output_paths[i][3] + '_' + hr
    EAST = output_paths[i][4] + '_' + hr
    
    #Calculate wind speed and save
    WindSpeed = RasterCalculator(rasters = [NORTH, EAST],
                                 input_names = ['N', 'E'],
                                 expression = 'SquareRoot(Square(N) + Square(E)))'
                                 )
    WindSpeed.save(output_paths[i][5] + '_' + hr)
    print(f'Wind Speed {methods[i]} complete')
    
    #Calculate wind direction and save
    WindDirection = RasterCalculator(rasters = [NORTH, EAST],
                                     input_names = ['N', 'E'],
                                     expression = 'ATan2(E, N)'
                                     )
    WindDirection.save(output_paths[i][6] + '_' + hr)
    print(f'Wind Direction {methods[i]} complete')

Wind Speed IDW1 complete
Wind Direction IDW1 complete
Wind Speed IDW2 complete
Wind Direction IDW2 complete
Wind Speed NN complete
Wind Direction NN complete
Wind Speed Spline complete
Wind Direction Spline complete
Wind Speed Kriging complete
Wind Direction Kriging complete
